In [429]:
import requests
import pandas as pd
#from bs4 import BeautifulSoup
from lxml import etree
import re


htmls='<script defer src="/min/ig-igv2-js-product.js?v=1665506897" type="text/javascript"></script>'
soup = BeautifulSoup(htmls, 'html.parser')
#url='https://www.instant-gaming.com/en/10612-buy-syberia-the-world-before-deluxe-edition-deluxe-edition-pc-game-steam/'
#soup = BeautifulSoup(url.read())
tag=soup.script
src=tag.attrs['src']
src

apa=src.replace('/min/ig-igv2-js-product.js?v=','')
int(apa)

e=int(''.join(filter(str.isdigit, src)))
e




In [430]:
url_endpoint = "https://www.instant-gaming.com/en/search/"

reqParams = {
    'q': 'Syberia'
}

In [431]:
reqParams = {
    'platform[]':1,
    'type[]':'steam',
    'sort_by':'bestsellers_desc',
    'min_reviewsavg':10,
    'max_reviewsavg':100,
    'noreviews':1,
    'min_price':0,
    'max_price':999,
    'noprice':1,
    'instock':1,
    'gametype':'games',
    'query':'syberia',
}

In [432]:
webpage = requests.get(url_endpoint, params=reqParams)
#webpage= requests.get(url2)
webpage.url

'https://www.instant-gaming.com/en/search/?platform%5B%5D=1&type%5B%5D=steam&sort_by=bestsellers_desc&min_reviewsavg=10&max_reviewsavg=100&noreviews=1&min_price=0&max_price=999&noprice=1&instock=1&gametype=games&query=syberia'

soup1 = BeautifulSoup(webpage.content, "html.parser")
soup1

dom = etree.HTML(str(soup1))

In [433]:
dom=etree.HTML(webpage.content)
dom

<Element html at 0x2c1e7499080>

In [434]:
tes=dom.xpath('/html/body/div[5]/div/div[1]/div/div/div/div/span/text()')
tes

['Syberia',
 'Syberia 3',
 'Syberia 2',
 'Syberia: The World Before Deluxe Edition',
 'Serious Sam: Siberian Mayhem']

In [435]:
price=dom.xpath('/html/body/div[5]/div/div[1]/div/div/div[2]/text()')
price

['11.00', '17.50', '11.00', '297.00', '143.94']

In [462]:
href=dom.xpath('/html/body/div[5]/div/div[1]/div/a/@href')
href

['https://www.instant-gaming.com/en/1244-buy-syberia-pc-mac-game-steam/',
 'https://www.instant-gaming.com/en/611-buy-syberia-3-pc-mac-game-steam/',
 'https://www.instant-gaming.com/en/1245-buy-syberia-2-pc-mac-game-steam/',
 'https://www.instant-gaming.com/en/10612-buy-syberia-the-world-before-deluxe-edition-deluxe-edition-pc-game-steam/',
 'https://www.instant-gaming.com/en/10229-buy-serious-sam-siberian-mayhem-pc-game-steam/']

In [461]:
tit=dom.xpath('/html/body/div[5]/div/div[1]/div/a/@title')
tit

['Syberia Steam 11.00<span>kr.</span>',
 'Syberia 3 Steam 17.50<span>kr.</span>',
 'Syberia 2 Steam 11.00<span>kr.</span>',
 'Syberia: The World Before Deluxe Edition Steam 297.00<span>kr.</span>',
 'Serious Sam: Siberian Mayhem Steam 143.94<span>kr.</span>']

In [460]:
ti=[]
for t in tit:
    t = re.sub(r'<.*?>', '', t)
    ti.append(t)
ti

['Syberia Steam 11.00kr.',
 'Syberia 3 Steam 17.50kr.',
 'Syberia 2 Steam 11.00kr.',
 'Syberia: The World Before Deluxe Edition Steam 297.00kr.',
 'Serious Sam: Siberian Mayhem Steam 143.94kr.']

In [488]:
df = pd.DataFrame(tes,columns=['name'])
df['platform'] = tit
df['urls'] = href
df['price'] = price

In [483]:
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    #result = re.sub('kr.','',result)
    return result
df['platform']=df['platform'].apply(lambda cw : remove_tags(cw))

In [489]:
df

,name,platform,urls,price
0,Syberia,Syberia Steam 11.00<span>kr.</span>,https://www.instant-gaming.com/en/1244-buy-syb...,11.00
1,Syberia 3,Syberia 3 Steam 17.50<span>kr.</span>,https://www.instant-gaming.com/en/611-buy-sybe...,17.50
2,Syberia 2,Syberia 2 Steam 11.00<span>kr.</span>,https://www.instant-gaming.com/en/1245-buy-syb...,11.00
3,Syberia: The World Before Deluxe Edition,Syberia: The World Before Deluxe Edition Steam...,https://www.instant-gaming.com/en/10612-buy-sy...,297.00
4,Serious Sam: Siberian Mayhem,Serious Sam: Siberian Mayhem Steam 143.94<span...,https://www.instant-gaming.com/en/10229-buy-se...,143.94


In [474]:
df['name']

0                                     Syberia
1                                   Syberia 3
2                                   Syberia 2
3    Syberia: The World Before Deluxe Edition
4                Serious Sam: Siberian Mayhem
Name: name, dtype: object

In [504]:
df["platform"]= df.apply(lambda x: x['platform'].replace(str(x['name']), ''), axis=1)
df["platform"]= df.apply(lambda x: x['platform'].replace(str(x['price']), ''), axis=1)
df['platform']=df.platform.str.replace(' kr.', '', regex=True)
df

,name,platform,urls,price
0,Syberia,Steam <span>kr.</span>,https://www.instant-gaming.com/en/1244-buy-syb...,11.00
1,Syberia 3,Steam <span>kr.</span>,https://www.instant-gaming.com/en/611-buy-sybe...,17.50
2,Syberia 2,Steam <span>kr.</span>,https://www.instant-gaming.com/en/1245-buy-syb...,11.00
3,Syberia: The World Before Deluxe Edition,Steam <span>kr.</span>,https://www.instant-gaming.com/en/10612-buy-sy...,297.00
4,Serious Sam: Siberian Mayhem,Steam <span>kr.</span>,https://www.instant-gaming.com/en/10229-buy-se...,143.94


df2 = df.replace({'platform': '<span>kr.</span>'}, 
    {'platform': ''}, regex=True)

df2 = df.apply(lambda x: x.replace({'Py':'Python with', 'Language':'Lang'}, regex=True))

In [501]:
df2 = df.replace({'platform': 'Syberia ', 'platform': '<span>kr.</span>'}, 
    {'platform': '', 'platform': ''}, regex=True)
df2

,name,platform,urls,price
0,Syberia,Syberia Steam 11.00,https://www.instant-gaming.com/en/1244-buy-syb...,11.00
1,Syberia 3,Syberia 3 Steam 17.50,https://www.instant-gaming.com/en/611-buy-sybe...,17.50
2,Syberia 2,Syberia 2 Steam 11.00,https://www.instant-gaming.com/en/1245-buy-syb...,11.00
3,Syberia: The World Before Deluxe Edition,Syberia: The World Before Deluxe Edition Steam...,https://www.instant-gaming.com/en/10612-buy-sy...,297.00
4,Serious Sam: Siberian Mayhem,Serious Sam: Siberian Mayhem Steam 143.94,https://www.instant-gaming.com/en/10229-buy-se...,143.94


soup2 = BeautifulSoup(etree.tostring(paths[0]), 'html.parser')
print (type(soup2))
print (soup.prettify())

mu=soup1.select('body > div.main-content > div > div > div > a')
for mui in mu:
    print(mui['href'])

mu